Optical Flow?
-  연속된 두 프레임 사이에서 픽셀이 어디로 움직였는지 계산
- 물체 속도 방향
- 자율주행, 드론, 동작인식

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow # Colab 환경에서 이미지 표시를 위한 함수 임포트
import time # 출력 속도를 조절하기 위한 time 모듈 임포트

# --- 1. 초기 설정 및 변수 정의 ---
# NOTE: Colab에서 이 코드를 실행하기 전에, 'newyork.mp4' 파일을 Colab 환경에 업로드해야 합니다.
video_path = '/home/kim/Desktop/AI_cours/data/newyork.mp4' # 처리할 비디오 파일 경로
cap = cv2.VideoCapture(video_path) # 비디오 캡처 객체 생성

# 비디오 파일이 제대로 열렸는지 확인
if not cap.isOpened():
    print("오류: 비디오 파일을 열 수 없습니다. 파일을 업로드했는지 확인하세요.")
    exit()

In [4]:
cv2.TERM_CRITERIA_EPS

2

In [5]:
cv2.TERM_CRITERIA_COUNT

1

In [6]:
1 | 2

3

In [3]:
(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)

(3, 10, 0.03)

In [7]:
# 프레임 재생 속도 조절을 위한 딜레이 (1000ms / 30fps = 약 33ms)
# Colab에서는 실시간 딜레이 대신 time.sleep()으로 대체되지만, 변수는 주석을 위해 유지합니다.

delay = int(1000/30)  # 33
# delay
# real-time 실시간 영상에서 부드러운 재생 원하기 때문에 필요한 타이밍

# 추적 경로를 그리기 위한 랜덤 색상(200개 코너점에 대응하는 색상)
# np.random.randint(0, 255, (200,3))
# 0-255 tkdldml 3chnnel(BGR) 색상 200개 생성

color = np.random.randint(0, 255, (200,3)) # 랜덤 색상 200개 생성

lines = None    # 추적선(이동경로) 그릴 이미지 저장 변수(초기화: 첫 프레임에서 진행)
prevImg = None  # previous image 이전 프레임 저장 변수(grayscale image)

# calcOpticalFlowPyrLK() 중지 요건 설정 (Termination Criteria)
# (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 최대 반복 횟수(10), 오차 임계값(0.03))
# cv2.TERM_CRITERIA_EPS : 움직임이 0.03 px 미만 >> 수렴 (error 오차가 충분히 작아지면 멈추는 조건)
# cv2.TERM_CRITERIA_COUNT, 최대 반복(iteration) 횟수(10)
termcriteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03) # |는 OR 비트 계산
# termcriteria # (3, 10, 0.03)

# 프레임 처리 개수 제한
frame_count = 0
MAX_FRAMES_TO_PROCESS = 150 # 최대 150 프레임만 처리
DISPLAY_EVERY_N_FRAMES = 20 # 20 프레임마다 결과 출력

print(f"광학 흐름 추적 시작 (최대 {MAX_FRAMES_TO_PROCESS} 프레임, {DISPLAY_EVERY_N_FRAMES} 프레임마다 출력)...")


광학 흐름 추적 시작 (최대 150 프레임, 20 프레임마다 출력)...


In [8]:
# 예제
'''
p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None,
                                      winSize=(15,15),
                                      maxLevel=2,
                                      criteria=termcriteria)

cv2.calcOpticalFlowPyrLK (루카스-카나데 광학 흐름) 물체 이동 위치를 추적
old_gray, gray (grayscale한 이미지) 이전사진, 현재 사진
p0 : position 추적하고 싶은 점들의 좌표
None :  기본 설정
winSize=(15,15) 추적한 윈도우 사이즈 (주변 15*15 영역 비교 이동방향 계산)
maxLevel=2 이미지 피라미드 레벨 (영상 축소 단계 수)
criteria=termcriteria : 우리가 설정한 정지조건(기준) (최대 10번 또는 오차 0.03 미만)

p1: 추적된 새로운 좌표값 (position)
st: 추적 성공 여부 (1: 성공, 0: 실패) status
err: 추적 오류 (오차 정도) error

p0 (100, 200) 서있는 상태
p1 (50, 100)  오리걸음 하는 상태

st = 1 # 성공 (잘 추적했네)
err = 0.01 # 오차 거의 없네 (기준인 0.03 미만이니깐)
'''

'\np1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None,\n                                      winSize=(15,15),\n                                      maxLevel=2,\n                                      criteria=termcriteria)\n\ncv2.calcOpticalFlowPyrLK (루카스-카나데 광학 흐름) 물체 이동 위치를 추적\nold_gray, gray (grayscale한 이미지) 이전사진, 현재 사진\np0 : position 추적하고 싶은 점들의 좌표\nNone :  기본 설정\nwinSize=(15,15) 추적한 윈도우 사이즈 (주변 15*15 영역 비교 이동방향 계산)\nmaxLevel=2 이미지 피라미드 레벨 (영상 축소 단계 수)\ncriteria=termcriteria : 우리가 설정한 정지조건(기준) (최대 10번 또는 오차 0.03 미만)\n\np1: 추적된 새로운 좌표값 (position)\nst: 추적 성공 여부 (1: 성공, 0: 실패) status\nerr: 추적 오류 (오차 정도) error\n\np0 (100, 200) 서있는 상태\np1 (50, 100)  오리걸음 하는 상태\n\nst = 1 # 성공 (잘 추적했네)\nerr = 0.01 # 오차 거의 없네 (기준인 0.03 미만이니깐)\n'

In [ ]:
# prevPt.shape

(182, 1, 2)

In [ ]:
# print(prevPt[0])
# print(nextPt[0])


[[1858.3547   486.16647]]
[[1858.3547   486.16647]]


In [ ]:
video_path = '/home/kim/Desktop/AI_cours/data/newyork.mp4' # 처리할 비디오 파일 경로
cap = cv2.VideoCapture(video_path) # 비디오 캡처 객체 생성

# 비디오 파일이 제대로 열렸는지 확인
if not cap.isOpened():
    print("오류: 비디오 파일을 열 수 없습니다. 파일을 업로드했는지 확인하세요.")
    exit()

lines = None    # 추적선(이동경로) 그릴 이미지 저장 변수(초기화: 첫 프레임에서 진행)
prevImg = None  # previous image 이전 프레임 저장 변수(grayscale image)
frame_count = 0
MAX_FRAMES_TO_PROCESS = 300 # 최대 150 프레임만 처리
DISPLAY_EVERY_N_FRAMES = 10 # 20 프레임마다 결과 출력

# 비디오 처리
while cap.isOpened() and frame_count < MAX_FRAMES_TO_PROCESS:
    ret, frame = cap.read() # 비디오에서 프레임 읽기

    if not ret:  # 프레임을 제대로 읽지 못했으면
       break

    # 현재 프레임 (사진) 복사 >> 추적결과를 그릴 이미지 준비
    img_draw = frame.copy()
    # optical flow 계산 위해 현재 프레임(사진)을 grayscale 로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 최초 프레임 처리(추적 시작)
    if prevImg is None:
        prevImg = gray  # 현재 gray 이미지를 '이전 이미지로' 저장
        # 추적선을 그릴 검은색 배경 이미지 생성 (원본 프레임과 동일 크기)
        lines = np.zeros_like(frame)
        # Shi-Tomasi 알고리즘으로 추적 시작할 코너점 200개 검출
        # (이전 이미지, 최대 코너점 수, 품질 임계값(0.01), 최소거리 10)
        prevPt = cv2.goodFeaturesToTrack(prevImg, 200, 0.01, 10) # 코너 200개 위치 반환
        print(prevPt.shape)
        # prevPt: 코너 점의 목록
    # 두번째 프레임 이후 처리(추적 진행 중)
    else:
        nextImg = gray # 현재 gray 이미지를 '다음 이미지로' 저장
        nextPt, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, # 
                                     prevPt,None, criteria=termcriteria)
        
        # 추적에 성공한 코너점(status == 1) 선별
        # (198, 1, 2) --> (198, 2) 차원이 바뀜
        # 그리고 점 개수도 성공 개수에 따라 줄음
        # 따라서 같은 점임에도 색상이 바뀜 인덱스가 땡겨지기 때문에
        prevMv = prevPt[status==1] # 이전 프레임에서 추적 성공한 점
        nextMv = nextPt[status==1] # 현재 프레임에 대응하는 점

        # 추적 성공한 모든 쌍에 대해서 반복
        for i, (p, n) in enumerate(zip(prevMv, nextMv)): # 각각의 점 한개 씩 반환 200번 또는 그 이하 반복
          #  코너점 좌표 추출(배열 구조 해제)
          px, py = p.ravel()
          nx, ny = n.ravel()

          # 이전 코너(p)와 새로운 코너(n) 사이에 추적선 그리기 (lines 이미지에 누적)
          cv2.line(lines, (int(px), int(py)), (int(nx), int(ny)), color[i % len(color)].tolist(), 2)
          # 시작점: 이전 위치(px, py), 끝점 (nx, ny)
          # color[i % len(color)].tolist() : 코너점 i 에서 할당된 랜덤 색

          # 새로운 코너(n)에 원형 점 그리기(img_draw 이미지에 매 프레임마다 표시)
          cv2.circle(img_draw, (int(nx), int(ny)), 2, color[i % len(color)].tolist(), -1)

        # 누적된 추적선이 그려진 lines 이미지와 현재 프레임(img_draw)을 합성
        # >> 추적 경로가 비디오 프레임 위에 나타나요
        img_draw = cv2.add(img_draw, lines)

        # 다음 루프 위해 현재 프레임(사진)과 코너점 >> 이전 변수로 이동
        prevImg = nextImg

        prevPt =nextMv.reshape(-1, 1, 2)
        # prevPt를 nextMv 형태로 맞춰 줘야 해요.
        # (N, 2) >> (N,1,2)
        # -1 자동계산


        # 원래 좌표 형식 [[x,y],[x,y],[x,y]]  shape (3,2) >>> (N,2)
        # OpenCV optical Flow 함수 입력 [[[x,y]],[[x,y]],[[x,y]]] shape (3,1,2) >> (N, 1, 2)

    # Colab 출력: 특정 간격의 프레임만 표시
    if frame_count % DISPLAY_EVERY_N_FRAMES == 0:
        print(f"\n--- Frame {frame_count} 광학 흐름 결과 ---")
        cv2.imshow('img',img_draw) # 추적 결과 프레임 표시
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break


    # 키 입력 처리 (원본 코드에 있던 부분. Colab에서는 작동하지 않으므로,
    # 프레임 수 제한으로 대체하며 주석 처리만 해둡니다.)
    # key = cv2.waitKey(delay)
    # if key == 27:    # ESC 키 (ASCII 27): 루프 종료
    #     break
    # elif key == 8:  # Backspace 키 (ASCII 8): 추적 이력 지우기
    #     prevImg = None

    frame_count += 1

# --- 3. 종료 및 정리 (수정 완료) ---
# 모든 OpenCV 창 닫기 (Colab에서는 필요 없음)
# cv2.destroyAllWindows()
# 비디오 캡처 객체 반환 및 해제
cap.release()
cv2.destroyAllWindows()

(200, 1, 2)

--- Frame 0 광학 흐름 결과 ---

--- Frame 10 광학 흐름 결과 ---

--- Frame 20 광학 흐름 결과 ---

--- Frame 30 광학 흐름 결과 ---

--- Frame 40 광학 흐름 결과 ---

--- Frame 50 광학 흐름 결과 ---

--- Frame 60 광학 흐름 결과 ---

--- Frame 70 광학 흐름 결과 ---

--- Frame 80 광학 흐름 결과 ---

--- Frame 90 광학 흐름 결과 ---

--- Frame 100 광학 흐름 결과 ---

--- Frame 110 광학 흐름 결과 ---

--- Frame 120 광학 흐름 결과 ---

--- Frame 130 광학 흐름 결과 ---

--- Frame 140 광학 흐름 결과 ---

--- Frame 150 광학 흐름 결과 ---

--- Frame 160 광학 흐름 결과 ---

--- Frame 170 광학 흐름 결과 ---

--- Frame 180 광학 흐름 결과 ---

--- Frame 190 광학 흐름 결과 ---

--- Frame 200 광학 흐름 결과 ---

--- Frame 210 광학 흐름 결과 ---

--- Frame 220 광학 흐름 결과 ---

--- Frame 230 광학 흐름 결과 ---

--- Frame 240 광학 흐름 결과 ---

--- Frame 250 광학 흐름 결과 ---

--- Frame 260 광학 흐름 결과 ---

--- Frame 270 광학 흐름 결과 ---

--- Frame 280 광학 흐름 결과 ---

--- Frame 290 광학 흐름 결과 ---
